In [1]:
# Import dependencies.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\benni\anaconda3\envs\PythonData\lib\site-packages

In [2]:
# Import the input dataset.
file_path = "Resources/charity_data.csv"
charity_df = pd.read_csv(file_path)
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# Data Preprocessing

In [3]:
# Get all columns
charity_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [4]:
# Check for null values
charity_df.isnull().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [5]:
# Check the unique value counts to see if binning is required
ask_amounts = charity_df.ASK_AMT.value_counts()
ask_amounts

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [6]:
# Determine max / min values of ask amount
charity_df.ASK_AMT.min()

5000

In [7]:
# Determine max / min values of ask amount
charity_df.ASK_AMT.max()

8597806340

In [8]:
charity_df.ASK_AMT.describe()

count    3.429900e+04
mean     2.769199e+06
std      8.713045e+07
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.742000e+03
max      8.597806e+09
Name: ASK_AMT, dtype: float64

In [9]:
# Determine which values to replace
replace_ask_amount = list(ask_amounts[ask_amounts < 1000].index)

# Replace in DataFrame
for amount in replace_ask_amount:
    charity_df.ASK_AMT = charity_df.ASK_AMT.replace(amount,"Other")


# Check to make sure binning was successful
charity_df.ASK_AMT.value_counts()

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [10]:
# Encode ASK_AMT
charity_df = pd.get_dummies(charity_df, columns=['ASK_AMT'])
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,1,1,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,0,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,0,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,0,1


In [11]:
# Check the unique value counts to see if binning is required
charity_df.IS_SUCCESSFUL.value_counts()

1    18261
0    16038
Name: IS_SUCCESSFUL, dtype: int64

In [12]:
# Check the unique value counts to see if binning is required
charity_df.STATUS.value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [13]:
# Drop rows where STATUS = 0
#charity_df = charity_df[charity_df['STATUS'] == 1]
#charity_df.STATUS.value_counts()

In [14]:
# Check the unique value counts to see if binning is required
application_types = charity_df.APPLICATION_TYPE.value_counts()
application_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Determine which values to replace
replace_application_type = list(application_types[application_types < 1000].index)

# Replace in DataFrame
for app_type in replace_application_type:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app_type,"Other")

# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Encode APPLICATION_TYPE
charity_df = pd.get_dummies(charity_df, columns=['APPLICATION_TYPE'])
charity_df.head()

,EIN,NAME,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,Independent,C1000,ProductDev,Association,1,0,N,1,1,0,1,0,0,0,0,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,0,1,0,0,1,0,0,0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,1,0,0,0,0,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,0,1,0,0,1,0,0,0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,0,1,0,0,1,0,0,0


In [17]:
# Check the unique value counts to see if binning is required
affiliations = charity_df.AFFILIATION.value_counts()
affiliations

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [18]:
# Determine which values to replace
replace_affiliations = list(affiliations[affiliations < 15000].index)

# Replace in DataFrame
for affiliation in replace_affiliations:
    charity_df.AFFILIATION = charity_df.AFFILIATION.replace(affiliation,"Other")

# Check to make sure binning was successful
charity_df.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [19]:
# Check the unique value counts to see if binning is required
classifications = charity_df.CLASSIFICATION.value_counts()
classifications

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1728        1
C1283        1
C1580        1
C1248        1
C4200        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [20]:
# Determine which values to replace
replace_classifications = list(classifications[classifications < 1800].index)

# Replace in DataFrame
for classification in replace_classifications:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classification,"Other")

# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [21]:
# Encode CLASSIFICATION
charity_df = pd.get_dummies(charity_df, columns=['CLASSIFICATION'])
charity_df.head()

,EIN,NAME,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_5000,...,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,Independent,ProductDev,Association,1,0,N,1,1,...,0,0,0,0,1,0,0,0,0,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,Independent,Preservation,Co-operative,1,1-9999,N,1,0,...,1,0,0,0,0,0,1,0,0,0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,CompanySponsored,ProductDev,Association,1,0,N,0,1,...,0,0,1,0,0,0,0,0,1,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,CompanySponsored,Preservation,Trust,1,10000-24999,N,1,0,...,1,0,0,0,0,0,1,0,0,0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,Independent,Heathcare,Trust,1,100000-499999,N,1,0,...,1,0,0,0,1,0,0,0,0,0


In [22]:
# Check the unique value counts to see if binning is required
use_cases = charity_df.USE_CASE.value_counts()
use_cases

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [23]:
# Determine which values to replace
replace_use_cases = list(use_cases[use_cases < 5000].index)

# Replace in DataFrame
for use_case in replace_use_cases:
    charity_df.USE_CASE = charity_df.USE_CASE.replace(use_case,"Other")

# Check to make sure binning was successful
charity_df.USE_CASE.value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

In [24]:
# Check the unique value counts to see if binning is required
organizations = charity_df.ORGANIZATION.value_counts()
organizations

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [25]:
# Determine which values to replace
replace_organizations = list(organizations[organizations < 10000].index)

# Replace in DataFrame
for org_type in replace_organizations:
    charity_df.ORGANIZATION = charity_df.ORGANIZATION.replace(org_type,"Other")

# Check to make sure binning was successful
charity_df.ORGANIZATION.value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

In [26]:
# Check the unique value counts to see if binning is required
incomes = charity_df.INCOME_AMT.value_counts()
incomes

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [27]:
# Check the unique value counts to see if binning is required
considerations = charity_df.SPECIAL_CONSIDERATIONS.value_counts()
considerations

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [28]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
AFFILIATION                   3
USE_CASE                      3
ORGANIZATION                  3
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [29]:
# drop unwanted columns
charity_df=charity_df.drop(['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'], axis=1)
charity_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,Independent,ProductDev,Association,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0
1,Independent,Preservation,Other,1-9999,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0
2,CompanySponsored,ProductDev,Association,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0
3,CompanySponsored,Preservation,Trust,10000-24999,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0
4,Independent,Other,Trust,100000-499999,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0


In [30]:
# Regenerate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
charity_df[charity_cat].nunique()

AFFILIATION     3
USE_CASE        3
ORGANIZATION    3
INCOME_AMT      9
dtype: int64

In [31]:
# Create a OneHotEncoder instance.
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder  using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame.
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Independent,AFFILIATION_Other,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Merge one-hot encoded features and drop the originals.
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,IS_SUCCESSFUL,ASK_AMT_5000,ASK_AMT_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,1,0,1,0,0,0,0,0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,1,0,0,1,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,1,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,1,0,0,1,0,0,0,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,1,0,0,1,0,0,0,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# Training and Testing Datasets

In [33]:
# Split the preprocessed data into features and target arrays.
y = charity_df['IS_SUCCESSFUL'].values
X = charity_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into training and testing dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [34]:
# Create a StandardScaler instance.
scaler = StandardScaler()

# Fit the StandardScaler.
X_scaler = scaler.fit(X_train)

# Scale the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Model 1: baseline attempt
- 2 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 17 neurons layer 2
- relu activation (hidden layers)
- sigmoid activation (output layer)
- 50 epochs

In [35]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 17

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 33)                1089      
_________________________________________________________________
dense_1 (Dense)              (None, 17)                578       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 18        
Total params: 1,685
Trainable params: 1,685
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# Evaluate the model using the test data
model_loss1, model_accuracy1 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss1}, Accuracy: {model_accuracy1}")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
25724/25724 [==============================] - 3s 106us/sample - loss: 0.5833 - acc: 0.7077
Epoch 2/50
25724/25724 [==============================] - 2s 91us/sample - loss: 0.5597 - acc: 0.7281
Epoch 3/50
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5561 - acc: 0.7284
Epoch 4/50
25724/25724 [==============================] - 2s 88us/sample - loss: 0.5544 - acc: 0.7296
Epoch 5/50
25724/25724 [==============================] - 2s 87us/sample - loss: 0.5523 - acc: 0.7299
Epoch 6/50
25724/25724 [==============================] - 2s 93us/sample - loss: 0.5512 - acc: 0.7311
Epoch 7/50
25724/25724 [==============================] - 3s 120us/sample - loss: 0.5507 - acc: 0.7296
Epoch 8/50
25724/25724 [==============================] - 2s 92us/sample - loss: 0.5502 - acc: 0.7316
Epoch 9/50
25724/25724 [==============================] - 2s 89us/sample - loss: 0.5493 

# Model 2: more epochs
- 2 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 17 neurons layer 2
- relu activation (hidden layers)
- sigmoid activation (output layer)
- 100 epochs

In [37]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 17

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 33)                1089      
_________________________________________________________________
dense_4 (Dense)              (None, 17)                578       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 18        
Total params: 1,685
Trainable params: 1,685
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

Epoch 1/100
25724/25724 [==============================] - 3s 111us/sample - loss: 0.5798 - acc: 0.7147
Epoch 2/100
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5581 - acc: 0.7274
Epoch 3/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5549 - acc: 0.7294
Epoch 4/100
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5535 - acc: 0.7310
Epoch 5/100
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5525 - acc: 0.7299
Epoch 6/100
25724/25724 [==============================] - 2s 96us/sample - loss: 0.5513 - acc: 0.7307
Epoch 7/100
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5507 - acc: 0.7313
Epoch 8/100
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5507 - acc: 0.7306
Epoch 9/100
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5497 - acc: 0.7329
Epoch 10/100
25724/25724 [==============================] - 3s 101us/s

# Model 3: more layers
- 3 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 17 neurons layer 2
- 9 neurons layer 3
- relu activation (hidden layers)
- sigmoid activation (output layer)
- 50 epochs

In [39]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 17
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 33)                1089      
_________________________________________________________________
dense_7 (Dense)              (None, 17)                578       
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 162       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 10        
Total params: 1,839
Trainable params: 1,839
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

Epoch 1/50
25724/25724 [==============================] - 3s 113us/sample - loss: 0.5798 - acc: 0.7152
Epoch 2/50
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5568 - acc: 0.7305
Epoch 3/50
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5541 - acc: 0.7300
Epoch 4/50
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5528 - acc: 0.7306
Epoch 5/50
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5516 - acc: 0.7313
Epoch 6/50
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5507 - acc: 0.7326
Epoch 7/50
25724/25724 [==============================] - 3s 101us/sample - loss: 0.5501 - acc: 0.7324
Epoch 8/50
25724/25724 [==============================] - 3s 105us/sample - loss: 0.5497 - acc: 0.7328
Epoch 9/50
25724/25724 [==============================] - 3s 97us/sample - loss: 0.5491 - acc: 0.7326
Epoch 10/50
25724/25724 [==============================] - 3s 99us/sample - los

# Model 4: change activation type of hidden layers
- 3 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 17 neurons layer 2
- 9 neurons layer 3
- tanh activation (hidden layers)
- sigmoid activation (output layer)
- 50 epochs

In [41]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 17
hidden_nodes_layer3 = 9

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 33)                1089      
_________________________________________________________________
dense_11 (Dense)             (None, 17)                578       
_________________________________________________________________
dense_12 (Dense)             (None, 9)                 162       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 10        
Total params: 1,839
Trainable params: 1,839
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# Evaluate the model using the test data
model_loss4, model_accuracy4 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss4}, Accuracy: {model_accuracy4}")

Epoch 1/50
25724/25724 [==============================] - 3s 114us/sample - loss: 0.5773 - acc: 0.7170
Epoch 2/50
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5598 - acc: 0.7280
Epoch 3/50
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5562 - acc: 0.7293
Epoch 4/50
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5542 - acc: 0.7301
Epoch 5/50
25724/25724 [==============================] - 2s 97us/sample - loss: 0.5536 - acc: 0.7310
Epoch 6/50
25724/25724 [==============================] - 3s 100us/sample - loss: 0.5520 - acc: 0.7316
Epoch 7/50
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5510 - acc: 0.7317
Epoch 8/50
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5505 - acc: 0.7314
Epoch 9/50
25724/25724 [==============================] - 3s 99us/sample - loss: 0.5495 - acc: 0.7298
Epoch 10/50
25724/25724 [==============================] - 3s 98us/sample - lo

# Model 5: more neurons
- 3 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 33 neurons layer 2
- 33 neurons layer 3
- tanh activation (hidden layers)
- sigmoid activation (output layer)
- 50 epochs

In [43]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 33
hidden_nodes_layer3 = 33

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 33)                1089      
_________________________________________________________________
dense_15 (Dense)             (None, 33)                1122      
_________________________________________________________________
dense_16 (Dense)             (None, 33)                1122      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 34        
Total params: 3,367
Trainable params: 3,367
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# Evaluate the model using the test data
model_loss5, model_accuracy5 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss5}, Accuracy: {model_accuracy5}")

Epoch 1/50
25724/25724 [==============================] - 3s 124us/sample - loss: 0.5760 - acc: 0.7182
Epoch 2/50
25724/25724 [==============================] - 3s 107us/sample - loss: 0.5610 - acc: 0.7277
Epoch 3/50
25724/25724 [==============================] - 3s 108us/sample - loss: 0.5584 - acc: 0.7276
Epoch 4/50
25724/25724 [==============================] - 3s 110us/sample - loss: 0.5549 - acc: 0.7285
Epoch 5/50
25724/25724 [==============================] - 3s 108us/sample - loss: 0.5537 - acc: 0.7286
Epoch 6/50
25724/25724 [==============================] - 2s 92us/sample - loss: 0.5524 - acc: 0.7305
Epoch 7/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5512 - acc: 0.7299
Epoch 8/50
25724/25724 [==============================] - 1s 53us/sample - loss: 0.5507 - acc: 0.7315
Epoch 9/50
25724/25724 [==============================] - 2s 60us/sample - loss: 0.5500 - acc: 0.7312
Epoch 10/50
25724/25724 [==============================] - 1s 51us/sample - l

# Model 6: more hidden layers
- 4 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 17 neurons layer 2
- 9 neurons layer 3
- 5 neurons layer 4
- tanh activation (hidden layers)
- sigmoid activation (output layer)
- 50 epochs

In [45]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 17
hidden_nodes_layer3 = 9
hidden_nodes_layer4 = 5

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 33)                1089      
_________________________________________________________________
dense_19 (Dense)             (None, 17)                578       
_________________________________________________________________
dense_20 (Dense)             (None, 9)                 162       
_________________________________________________________________
dense_21 (Dense)             (None, 5)                 50        
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 6         
Total params: 1,885
Trainable params: 1,885
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# Evaluate the model using the test data
model_loss6, model_accuracy6 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss6}, Accuracy: {model_accuracy6}")

Epoch 1/50
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5841 - acc: 0.7101
Epoch 2/50
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5614 - acc: 0.7290
Epoch 3/50
25724/25724 [==============================] - 1s 44us/sample - loss: 0.5576 - acc: 0.7293
Epoch 4/50
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5552 - acc: 0.7311
Epoch 5/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5531 - acc: 0.7313
Epoch 6/50
25724/25724 [==============================] - 1s 42us/sample - loss: 0.5523 - acc: 0.7311
Epoch 7/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5512 - acc: 0.7319
Epoch 8/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5502 - acc: 0.7319
Epoch 9/50
25724/25724 [==============================] - 1s 43us/sample - loss: 0.5495 - acc: 0.7319
Epoch 10/50
25724/25724 [==============================] - 1s 43us/sample - loss: 

# Model 7: more neurons
- 4 hidden layers
- 33 neurons (representing 33 columns in the dataframe) layer 1
- 33 neurons layer 2
- 33 neurons layer 3
- 33 neurons layer 4
- tanh activation (hidden layers)
- sigmoid activation (output layer)
- 50 epochs

In [47]:
# Define the model.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  33
hidden_nodes_layer2 = 33
hidden_nodes_layer3 = 33
hidden_nodes_layer4 = 33

nn = tf.keras.models.Sequential()

# Add hidden layers.
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Add the output layer.
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model.
nn.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 33)                1089      
_________________________________________________________________
dense_24 (Dense)             (None, 33)                1122      
_________________________________________________________________
dense_25 (Dense)             (None, 33)                1122      
_________________________________________________________________
dense_26 (Dense)             (None, 33)                1122      
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 34        
Total params: 4,489
Trainable params: 4,489
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

# Evaluate the model using the test data
model_loss7, model_accuracy7 = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss7}, Accuracy: {model_accuracy7}")

Epoch 1/50
25724/25724 [==============================] - 1s 54us/sample - loss: 0.5727 - acc: 0.7213
Epoch 2/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5597 - acc: 0.7281
Epoch 3/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5564 - acc: 0.7295
Epoch 4/50
25724/25724 [==============================] - 1s 49us/sample - loss: 0.5542 - acc: 0.7304
Epoch 5/50
25724/25724 [==============================] - 1s 47us/sample - loss: 0.5524 - acc: 0.7317
Epoch 6/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5515 - acc: 0.7318
Epoch 7/50
25724/25724 [==============================] - 1s 51us/sample - loss: 0.5499 - acc: 0.7310
Epoch 8/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5495 - acc: 0.7317
Epoch 9/50
25724/25724 [==============================] - 1s 50us/sample - loss: 0.5487 - acc: 0.7343
Epoch 10/50
25724/25724 [==============================] - 1s 50us/sample - loss: 

# Comparison of models' accuracy

In [49]:
# Show accuracy of each model
print(f"Model 1: accuracy = {model_accuracy1} \nModel 2: accuracy = {model_accuracy2} \nModel 3: accuracy = {model_accuracy3} \nModel 4: accuracy = {model_accuracy4} \nModel 5: accuracy = {model_accuracy5} \nModel 6: accuracy = {model_accuracy6} \nModel 7: accuracy = {model_accuracy7}")

Model 1: accuracy = 0.7308454513549805 
Model 2: accuracy = 0.729912519454956 
Model 3: accuracy = 0.7282798886299133 
Model 4: accuracy = 0.728863000869751 
Model 5: accuracy = 0.7264139652252197 
Model 6: accuracy = 0.7293294668197632 
Model 7: accuracy = 0.7321282625198364


# Model 8: Random Forest Classifier

In [50]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=256, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Model 8 accuracy: {accuracy_score(y_test,y_pred)}")

 Model 8 accuracy: 0.7315451895043732
